In [1]:
import tensorflow as tf
from tensorflow import keras
import h5py
import scipy.stats
import numpy as np
import os
import pandas as pd
from sklearn import model_selection
import glob
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

2024-02-20 15:43:21.760729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
result_df = pd.DataFrame(columns=['Model',"Pearson's R",'Cell Type'])
celltype_list = ['K562','HepG2']

model_list=[]
pr_list = []
ct_list = []

In [3]:
## One-hot input models
for ct in celltype_list:

    f = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/'+ct+'_onehot.h5', 'r')
    x = f['onehot'][()]
    y = f['target'][()]
    x = np.swapaxes(x,1,2)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.1,random_state=42)

    for model_f in glob.glob('../model/lenti_MPRA/lenti_MPRA_onehot/'+ct+'/*.h5'):
        model = tf.keras.models.load_model(model_f)
        y_pred = model.predict(x_test)
        pr = scipy.stats.pearsonr(np.squeeze(y_pred),y_test)[0]
        model_name = model_f.split('/')[-1][:-3]
        model_list.append(model_name)
        pr_list.append(pr)
        ct_list.append(ct)

#result_df = pd.DataFrame({'Model':model_list,"Pearson's R":pr_list,'Cell Type':ct_list})

2024-02-05 23:15:01.743028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:85:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2024-02-05 23:15:03.040175: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-05 23:15:03.136339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


438/438 [==============================] - 1s 969us/step


In [4]:
## NT saved results
for ct in celltype_list:
    temp_df = pd.read_csv('./results/'+ct+'_2B5_1000G_layersearch.csv')
    best_perf = temp_df['Embedding CNN'].max()
    model_list.append('NT')
    pr_list.append(best_perf)
    ct_list.append(ct)


In [5]:
## Embedding results
for model_name in ['gpn','hyena','sei','dnabert2']:
    for ct in celltype_list:
        f = h5py.File('../data/lenti_MPRA_embed/'+model_name+'_'+ct+'.h5', 'r')
        x = f['seq']
        y = f['mean']
        x_train, x_test, y_train, y_test = model_selection.train_test_split(range(len(x)), range(len(y)), 
                                                                            test_size=0.1,random_state=42)
        x_test = x[np.sort(x_test)]
        y_test = y[np.sort(y_test)]
       
        model = tf.keras.models.load_model('../model/lenti_MPRA/lenti_MPRA_embed/'+ct+'/'+model_name+'.h5')
        y_pred = model.predict(x_test)
        pr = scipy.stats.pearsonr(np.squeeze(y_pred),y_test)[0]
        model_list.append(model_name)
        pr_list.append(pr)
        ct_list.append(ct)

/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


438/438 [==============================] - 1s 2ms/step


In [7]:
result_df.to_csv('./results/lenti.csv',index=False)